## 1.注意力机制


### MHA

In [ ]:
import torch
import math 
class multiheadattention(torch.nn.Module):
    def __init__(self, d, h):
        super().__init__()
        self.d = d
        self.h = h
        self.k = d // h

        self.wqkv = torch.nn.Linear(d, d*3)
        self.wo = torch.nn.Linear(d, d)
    
    def forward(self, x, mask = None):
        B, L, D = x.shape
        qkv = self.wqkv(x)
        q, k, v = torch.chunk(qkv, 3, -1)
        # 可以用reshap代替transpose
        q = q.view(B, L, self.h, self.k).transpose(1, 2)
        k = k.view(B, L, self.h, self.k).transpose(1, 2)
        v = v.view(B, L, self.h, self.k).transpose(1, 2)

        attention_score =torch.matmul(q, k.transpose(-1,-2)) / math.sqrt(self.k)
        if mask is not None:
            attention_score = torch.masked_fill(attention_score, mask=mask, value=-1e9)
        attention_weight = torch.softmax(attention_score, dim=-1)
        context = torch.matmul(attention_weight, v).transpose(1, 2).contiguous().view(B, L, D)
        output = self.wo(context)
        return output, attention_weight
    
# --- 测试代码 ---
batch_size = 5
max_seq_len = 10
d_model = 64
head = 4

x = torch.randn(batch_size, max_seq_len, d_model)

attention_model = multiheadattention(d_model, head)
output, attention = attention_model(x) 

print("代码运行成功！")
print("输出张量的形状:", output.shape)
print("注意力权重的形状:", attention.shape)

代码运行成功！
输出张量的形状: torch.Size([5, 10, 64])
注意力权重的形状: torch.Size([5, 4, 10, 10])


### MQA

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiQueryAttention(nn.Module):
    """
    与原 MultiHeadAttention 的差异：
    - Q: 维持与 MHA 相同，shape -> [B, h, L, d_k]
    - K/V: 仅产生 1 组共享头，shape -> [B, 1, L, d_k]
    这样在推理时 KV cache 只需缓存 1 份（而非 h 份）。
    """
    def __init__(self, d_model, num_head):
        super(MultiQueryAttention, self).__init__()
        assert d_model % num_head == 0, "d_model 必须能被 num_head 整除"
        self.d_model = d_model
        self.num_head = num_head
        self.d_k = d_model // num_head

        # 与原实现的区别：
        #   - Q 仍然映射到 d_model（然后 reshape 成 h 个头）
        #   - K/V 只映射到 d_k（单头维度），且各 1 份
        self.wq = nn.Linear(d_model, d_model)      # 生成多头的 Q
        self.wkv = nn.Linear(d_model, 2 * self.d_k)  # 生成共享的 K、V（仅 1 头的维度）

        self.wo = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        B, L, _ = x.shape

        # 1) 计算 Q、K、V
        q = self.wq(x)                       # [B, L, d_model]
        kv = self.wkv(x)                     # [B, L, 2*d_k]
        k, v = torch.chunk(kv, 2, dim=-1)    # [B, L, d_k], [B, L, d_k]

        # 2) 形状整理
        # Q: [B, L, h, d_k] -> [B, h, L, d_k]
        q = q.view(B, L, self.num_head, self.d_k).transpose(1, 2)  # [B, h, L, d_k]

        # 共享 K/V：加一个“伪 head 维”=1，方便广播到 h
        # K/V: [B, L, d_k] -> [B, 1, L, d_k]
        k = k.unsqueeze(1)  # [B, 1, L, d_k]
        v = v.unsqueeze(1)  # [B, 1, L, d_k]

        # 3) 注意力分数：Q 与共享 K
        # scores: [B, h, L, L]，这里利用了 K 在 head 维度上的广播
        scores = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(self.d_k)  # [B, h, L, L]

        if mask is not None:
            # 要求 mask 能广播到 [B, h, L, L]
            # 例如 mask 形状可为 [B, 1, 1, L]（causal/ padding），或 [B, 1, L, L]
            scores = scores.masked_fill(mask, -1e9)

        attn = torch.softmax(scores, dim=-1)         # [B, h, L, L]

        # 4) 加权求和：与共享 V 相乘（同样通过广播）
        context = torch.matmul(attn, v)              # [B, h, L, d_k]

        # 5) 还原回 [B, L, d_model] 并输出
        context = context.transpose(1, 2).contiguous()         # [B, L, h, d_k]
        context = context.view(B, L, self.d_model)             # [B, L, d_model]
        output = self.wo(context)                               # [B, L, d_model]

        return output, attn


# --- 简单测试（与原 MHA 测试保持一致） ---
if __name__ == "__main__":
    batch_size = 2
    d_model = 10
    head = 2
    max_seq_len = 5
    x = torch.randn(batch_size, max_seq_len, d_model)

    attention_model = MultiQueryAttention(d_model, head)
    output, attention = attention_model(x)

    print("代码运行成功！（MQA）")
    print("输出张量的形状:", output.shape)     # 期望: [B, L, d_model]
    print("注意力权重的形状:", attention.shape)  # 期望: [B, h, L, L]


### GQA

In [29]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class GroupedQueryAttention(nn.Module):
    """
    GQA: 把 h 个 Query 头分成 g 组；组内共享 1 套 K/V
    - num_head = h
    - num_kv_head = g (1 < g <= h, 且 h % g == 0)
    形状约定：
      Q: [B, h, L, d_k]
      K,V(分组): [B, g, L, d_k]
    计算时把 Q reshape 成 [B, g, h_per_group, L, d_k]，
    与同组的 K,V 做注意力；最后再还原回 [B, h, L, d_k]。
    """
    def __init__(self, d_model, num_head, num_kv_head):
        super(GroupedQueryAttention, self).__init__()
        assert d_model % num_head == 0, "d_model 必须能被 num_head 整除"
        assert 1 <= num_kv_head <= num_head, "num_kv_head 必须在 [1, num_head] 范围内"
        assert num_head % num_kv_head == 0, "num_head 必须能被 num_kv_head 整除（每组等量分配）"

        self.d_model = d_model
        self.num_head = num_head            # h
        self.num_kv_head = num_kv_head      # g
        self.d_k = d_model // num_head
        self.h_per_group = self.num_head // self.num_kv_head  # h/g

        # Q 仍映射到 d_model（随后 reshape 为 h 个头）
        self.wq = nn.Linear(d_model, d_model)

        # K/V 只映射到 g * d_k（随后 reshape 为 g 个“KV 头”）
        self.wkv = nn.Linear(d_model, 2 * self.num_kv_head * self.d_k)

        self.wo = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        B, L, _ = x.shape

        # 1) 投影
        # Q: [B, L, d_model]
        # KV: [B, L, 2 * g * d_k] -> split -> [B, L, g * d_k] 各自
        q = self.wq(x)
        kv = self.wkv(x)
        k, v = torch.chunk(kv, 2, dim=-1)

        # 2) 形状整理
        # Q -> [B, h, L, d_k]
        q = q.view(B, L, self.num_head, self.d_k).transpose(1, 2)

        # K,V -> [B, g, L, d_k]
        k = k.view(B, L, self.num_kv_head, self.d_k).transpose(1, 2)
        v = v.view(B, L, self.num_kv_head, self.d_k).transpose(1, 2)

        # 把 Q 分组： [B, h, L, d_k] -> [B, g, h_per_group, L, d_k]
        qg = q.view(B, self.num_kv_head, self.h_per_group, L, self.d_k)

        # 为了与组内 K,V 做 batched matmul，给 K,V 加一个组内头维度=1，方便广播
        # Kg, Vg: [B, g, 1, L, d_k]
        Kg = k.unsqueeze(2)
        Vg = v.unsqueeze(2)

        # 3) 组内注意力分数： [B, g, h_per_group, L, L]
        # 等价于：scores_g[b,g,hg] = qg[b,g,hg] @ Kg[b,g,0]^T / sqrt(d_k)
        scores_g = torch.matmul(qg, Kg.transpose(-1, -2)) / math.sqrt(self.d_k)

        if mask is not None:
            # 要求 mask 能广播到 [B, 1 或 g, 1 或 h_per_group, L, L] 或最终 [B, h, L, L]
            # 最常见做法：提供 [B, 1, 1, L, L]（或 [B, 1, 1, 1, L] 的causal/pad组合）
            scores_g = scores_g.masked_fill(mask == 0, -1e9)

        attn_g = torch.softmax(scores_g, dim=-1)               # [B, g, h_per_group, L, L]

        # 4) 组内加权求和：context_g: [B, g, h_per_group, L, d_k]
        context_g = torch.matmul(attn_g, Vg)

        # 5) 还原回所有头：先合并 g 与 h_per_group -> h
        context = context_g.reshape(B, self.num_head, L, self.d_k)  # [B, h, L, d_k]
        context = context.transpose(1, 2).contiguous()              # [B, L, h, d_k]
        context = context.view(B, L, self.d_model)                  # [B, L, d_model]
        output = self.wo(context)

        # 同样给出注意力权重（按头展平回 [B, h, L, L]，便于对齐可视化）
        attn = attn_g.reshape(B, self.num_head, L, L)

        return output, attn


# --- 简单测试（与原 MHA 测试风格一致） ---
if __name__ == "__main__":
    batch_size = 2
    d_model = 12
    num_head = 6     # h
    num_kv_head = 3  # g（每组 2 个 Q 头）
    max_seq_len = 5

    x = torch.randn(batch_size, max_seq_len, d_model)
    gqa = GroupedQueryAttention(d_model, num_head, num_kv_head)
    out, attn = gqa(x)

    print("代码运行成功！（GQA）")
    print("输出张量形状:", out.shape)      # 期望: [B, L, d_model]
    print("注意力形状  :", attn.shape)     # 期望: [B, h, L, L]


代码运行成功！（GQA）
输出张量形状: torch.Size([2, 5, 12])
注意力形状  : torch.Size([2, 6, 5, 5])


## 2.AUC

### AUC
基于排序（Rank / Mann–Whitney U）的 AUC 计算方法。

输入：
- labels: List[int] 或 1D numpy array
    样本真实标签，取值为 {0, 1}
- scores: List[float] 或 1D numpy array
    模型预测分数，分数越大表示越可能为正样本

输出：
- auc: float
    AUC 值，取值范围 [0, 1]

核心思想：
1. 按预测分数从小到大排序
2. 扫描排序后的样本序列
3. 每遇到一个正样本，统计其前面已有多少负样本
    这些负样本都被该正样本“正确地排在后面”

In [ ]:
import numpy as np
 
def auc_rank(labels, scores):
 
    # 转为 numpy array，便于排序和向量化操作
    labels = np.asarray(labels)
    scores = np.asarray(scores)
    print(f"labels:{labels}")
    print(f"scores:{scores}")

 
    # 获取按照 score 从小到大排序后的索引
    order = np.argsort(scores)
    print(f"order:{order}")
 
    # 按排序后的顺序重排标签
    labels_sorted = labels[order]
    print(f"labels_sorted:{labels_sorted}")
 
 
    # 正样本数量 |P|
    n_pos = np.sum(labels_sorted == 1)
    print(f"n_pos:{n_pos}")
 
    # 负样本数量 |N|
    n_neg = np.sum(labels_sorted == 0)
    print(f"n_neg:{n_neg}")
    # 已扫描到的负样本数量（前缀负样本计数）
    neg_count = 0
 
    # 排序正确的正负样本对数量
    correct = 0.0
 
    # 从低分到高分扫描
    for l in labels_sorted:
        if l == 1:
            # 当前是正样本：
            # 它前面的所有负样本都满足 score_neg < score_pos
            correct += neg_count
        else:
            # 当前是负样本，增加负样本计数
            neg_count += 1
 
    # AUC = 排序正确的正负样本对 / 总正负样本对
    return correct / (n_pos * n_neg)

label = [0, 1, 0, 0, 1, 0, 0, 1]
q = [0.1, 0.9, 0.2, 0.8, 1, 0.2, 0.3, 0.8]
auc  = auc_rank(label, q)
print(f"auc:{auc}")

### GAUC
基于排序的 GAUC 计算方法(Group AUC)。

GAUC 通过在每个用户内部分别计算 AUC,然后进行加权平均,
从而消除用户间基线差异的影响,更准确地评估模型的用户内排序能力。

输入:
- user_ids: List[str] 或 1D numpy array
    每个样本对应的用户标识
- labels: List[int] 或 1D numpy array  
    样本真实标签,取值为 {0, 1}
- scores: List[float] 或 1D numpy array
    模型预测分数,分数越大表示越可能为正样本
- weight_type: str, 默认 'impression'
    权重类型,可选值:
    - 'impression': 按用户曝光次数加权(工业界常用)
    - 'uniform': 等权重,每个用户权重为 1
    
输出:
- gauc: float
    加权后的 GAUC 值,取值范围 [0, 1]
- user_auc_dict: dict
    每个用户的 AUC 值字典,用于分析不同用户的排序质量
    
核心思想:
1. 将样本按 user_id 分组
2. 在每个用户内部,使用 Rank-based 方法计算 AUC
3. 根据指定的权重类型对所有用户的 AUC 进行加权平均

In [28]:
import numpy as np
from collections import defaultdict
 
def gauc_rank(user_ids, labels, scores, weight_type='impression'):
    
    # 转为 numpy array
    user_ids = np.asarray(user_ids)
    labels = np.asarray(labels)
    scores = np.asarray(scores)
    
    # 按用户分组:构建 user_id -> 样本索引列表 的映射
    user_sample_dict = defaultdict(list)
    for idx, user_id in enumerate(user_ids):
        user_sample_dict[user_id].append(idx)
    
    # 存储每个用户的 AUC 和权重
    user_auc_dict = {}
    total_weighted_auc = 0.0
    total_weight = 0.0
    
    # 遍历每个用户,计算其 AUC
    for user_id, sample_indices in user_sample_dict.items():
        # 提取该用户的所有样本,user_id=1,sample_indices= [0, 1, 2, 3]
        user_labels = labels[sample_indices]
        user_scores = scores[sample_indices]
        
        # 计算该用户内的正负样本数
        n_pos = np.sum(user_labels == 1)
        n_neg = np.sum(user_labels == 0)
        
        # 如果该用户只有正样本或只有负样本,无法计算 AUC
        # 跳过该用户(也可以选择赋值为 0.5)
        if n_pos == 0 or n_neg == 0:
            continue
            
        # 使用 Rank-based 方法计算该用户的 AUC
        # 按 score 从小到大排序
        order = np.argsort(user_scores)
        sorted_labels = user_labels[order]
        
        # 统计排序正确的正负样本对数
        neg_count = 0
        correct_pairs = 0.0
        
        for label in sorted_labels:
            if label == 1:
                # 正样本:其前面的所有负样本都被正确排序
                correct_pairs += neg_count
            else:
                # 负样本:计数增加
                neg_count += 1
        
        # 该用户的 AUC
        user_auc = correct_pairs / (n_pos * n_neg)
        user_auc_dict[user_id] = user_auc
        
        # 确定该用户的权重
        if weight_type == 'impression':
            # 按曝光次数加权
            weight = len(sample_indices)
        elif weight_type == 'uniform':
            # 等权重
            weight = 1.0
        else:
            raise ValueError(f"Unknown weight_type: {weight_type}")
        
        # 累加加权 AUC
        total_weighted_auc += user_auc * weight
        total_weight += weight
    
    # 计算 GAUC
    if total_weight == 0:
        return 0.5, user_auc_dict
    
    gauc = total_weighted_auc / total_weight
    print(f"gauc:{gauc}")
    print(f"user_auc_dict:{user_auc_dict}")
    return gauc, user_auc_dict

label = [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0]
q = [0.1, 0.9, 0.2, 0.8, 1, 0.2, 0.3, 0.9, 0.7, 0.9, 0.7]
user_id = [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3]
_, _ = gauc_rank(user_id, label, q, weight_type='impression')


gauc:0.8636363636363636
user_auc_dict:{1: 1.0, 2: 1.0, 3: 0.5}


## 损失函数

### BCE二类交叉熵实现

In [31]:
import numpy as np
 
class BCE_Logits_Loss:
    """
    一个数值稳定的、基于 Logits 的二元交叉熵损失函数实现。
    """
    def __init__(self, reduction = 'mean'):

        if reduction not in ['mean', 'sum', 'none']:
            raise ValueError("reduction 必须是 'mean', 'sum', 或 'none'")
        self.reduction = reduction
    
    def __call__(self, y_pred_logits: np.ndarray, y_true: np.ndarray):
        x = y_pred_logits
        y = y_true
 
        # 直接套用数值稳定的BCE公式
        per_sample_loss = np.maximum(x, 0) - x * y + np.log(1 + np.exp(-np.abs(x)))
 
        if self.reduction == "mean":
            return np.mean(per_sample_loss)
        elif self.reduction == "sum":
            return np.sum(per_sample_loss)
        else: # self.reduction == 'none'
            return per_sample_loss
        
x = np.array([5,-4,5,-6])
y = np.array([1,0,1,0])

mse_loss = BCE_Logits_Loss(reduction='mean')
loss = mse_loss(x,y)

print(f"bce损失为{loss}")

bce损失为0.008514077508444018


### MSE

In [30]:
import numpy as np
 
class MSE_Loss:
    """
    一个数值稳定的、基于 Logits 的二元交叉熵损失函数实现。
    """
    def __init__(self, reduction = 'mean'):

        if reduction not in ['mean', 'sum', 'none']:
            raise ValueError("reduction 必须是 'mean', 'sum', 或 'none'")
        self.reduction = reduction
    
    def __call__(self, y_pred_logits: np.ndarray, y_true: np.ndarray):
        
        x = y_pred_logits
        y = y_true
 
        # 直接套用数值稳定的BCE公式
        per_sample_loss = (y-x)**2
 
        if self.reduction == "mean":
            return np.mean(per_sample_loss)
        elif self.reduction == "sum":
            return np.sum(per_sample_loss)
        else: # self.reduction == 'none'
            return per_sample_loss
        
x = np.array([1,2,3,4])
y = np.array([1,2,4,4])

mse_loss = MSE_Loss(reduction='mean')
loss = mse_loss(x,y)

print(f"mse损失为{loss}")

mse损失为0.25


### focal_Loss

In [ ]:
import numpy as np

class FocalLoss:
    """
    一个简洁且高效的 Focal Loss 实现，用于处理类别不平衡问题。
    
    该实现接收模型输出的概率 (经过 Sigmoid 激活后) 作为输入。
    """
    def __init__(self, alpha: float = 0.25, gamma: float = 2.0, 
                 reduction = 'mean', eps: float = 1e-9):
        """
        初始化 Focal Loss。

        参数:
            alpha (float): 平衡参数，用于调节正负样本的权重。论文推荐 0.25。
            gamma (float): 聚焦参数，用于动态调整样本权重，使模型聚焦于难分样本。论文推荐 2.0。
            reduction (str): 指定损失的聚合方式 ('mean', 'sum', 'none')。
            eps (float): 一个极小的数值，用于避免 log(0) 导致的计算溢出。
        """
        if reduction not in ['mean', 'sum', 'none']:
            raise ValueError("reduction 必须是 'mean', 'sum', 或 'none'")
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.eps = eps
    
    def __call__(self, y_pred_prob: np.ndarray, y_true: np.ndarray) -> np.ndarray | float:

        # 1. 裁剪概率值以保证数值稳定性
        p = np.clip(y_pred_prob, self.eps, 1.0 - self.eps)

        # 2. 计算正负样本的损失项，并用 y_true 和 (1-y_true) 作为掩码
        # 正样本 (y_true=1) 的损失部分
        loss_pos = -self.alpha * ((1 - p) ** self.gamma) * np.log(p) * y_true
        
        # 负样本 (y_true=0) 的损失部分
        loss_neg = -(1 - self.alpha) * (p ** self.gamma) * np.log(1 - p) * (1 - y_true)

        # 3. 将两部分损失相加得到每个样本的最终损失
        per_sample_loss = loss_pos + loss_neg

        # 4. 根据指定的 reduction 策略聚合损失
        if self.reduction == "mean":
            return np.mean(per_sample_loss)
        elif self.reduction == "sum":
            return np.sum(per_sample_loss)
        else: # self.reduction == 'none'
            return per_sample_loss

x = np.array([0.9,0.1,0.9,0.7])
y = np.array([1,0,1,0])

mse_loss = FocalLoss(reduction='mean')
loss = mse_loss(x,y)

print(f"focal_bce损失为{loss}")

## 手写MLP

In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    """
    多层感知机（MLP）的完整实现
    Args:
        input_dim (int): 输入特征维度
        hidden_dims (list): 隐藏层维度列表，每个元素代表对应隐藏层的神经元数量
        output_dim (int): 输出维度
        activation (str): 激活函数类型，支持 'relu', 'tanh', 'sigmoid'
        dropout_rate (float): Dropout概率，用于正则化
        batch_norm (bool): 是否使用批归一化
    """
    
    def __init__(self, input_dim, hidden_dims, output_dim, 
                 activation='relu', dropout_rate=0.0, batch_norm=False):
        super(MLP, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.activation_name = activation
        self.dropout_rate = dropout_rate
        self.batch_norm = batch_norm
        
        # 构建网络层
        layers = []
        layer_dims = [input_dim] + hidden_dims + [output_dim]
        
        # 逐层构建网络
        for i in range(len(layer_dims) - 1):
            # 线性变换层
            linear_layer = nn.Linear(layer_dims[i], layer_dims[i + 1])
            layers.append(linear_layer)
            
            # 如果不是最后一层，添加激活函数、批归一化和dropout
            if i < len(layer_dims) - 2:
                # 批归一化（在激活函数之前）
                if self.batch_norm:
                    layers.append(nn.BatchNorm1d(layer_dims[i + 1]))
                
                # 激活函数
                if activation == 'relu':
                    layers.append(nn.ReLU())
                elif activation == 'tanh':
                    layers.append(nn.Tanh())
                elif activation == 'sigmoid':
                    layers.append(nn.Sigmoid())
                else:
                    raise ValueError(f"不支持的激活函数: {activation}")
                
                # Dropout正则化
                if dropout_rate > 0:
                    layers.append(nn.Dropout(dropout_rate))
        
        # 将所有层组合成Sequential模块
        self.network = nn.Sequential(*layers)
        
        # 初始化网络参数
        self._initialize_weights()
    
    def _initialize_weights(self):
        """
        网络参数初始化
        使用Xavier/Glorot初始化方法，这对于深层网络的训练非常重要
        """
        for module in self.modules():
            if isinstance(module, nn.Linear):
                # Xavier均匀分布初始化
                nn.init.xavier_uniform_(module.weight)
                # 偏置项初始化为0
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
    
    def forward(self, x):
        """
        前向传播过程
        
        Args:
            x (torch.Tensor): 输入张量，形状为 (batch_size, input_dim)
        
        Returns:
            torch.Tensor: 输出张量，形状为 (batch_size, output_dim)
        """
        return self.network(x)

In [34]:
import torch
import torch.nn as nn
# 常见顺序是：Linear -> LayerNorm -> Activation -> Dropout
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, activation='relu', dropout=0.1, layernorm=True):
        super().__init__()

        layer_dims = [input_dim] + hidden_dims + [output_dim]
        layers = []

        for i in range(len(layer_dims) - 1):
            layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))

            if i < len(layer_dims) - 2:  # 最后一层不要激活/Dropout/Norm
                if layernorm:
                    layers.append(nn.LayerNorm(layer_dims[i+1]))      # nn.LayerNorm 必须指定 特征维度大小（即输入最后一维的大小）

                if activation == 'relu':
                    layers.append(nn.ReLU())
                elif activation == 'sigmoid':
                    layers.append(nn.Sigmoid())
                elif activation == 'tanh':
                    layers.append(nn.Tanh())

                # Dropout接收的输入是经过ReLU激活后的向量，然后对这些激活值进行随机屏蔽，直接影响的是传递给下一层的信息
                if dropout > 0:
                    layers.append(nn.Dropout(dropout))

        # *被称为"解包操作符"（unpacking operator），它的作用是将一个可迭代对象（如列表、元组）中的元素逐个取出，作为独立的参数传递给函数。
        self.network = nn.Sequential(*layers)
        self.initialize_weight()

    def initialize_weight(self):
        for module in self.modules():       # 调用self.modules()会依次返回整个MLP模型、第一个Linear层、LayerNorm层、ReLU层、第二个Linear层等等
            if isinstance(module, nn.Linear):       # Linear层有权重矩阵和偏置向量需要初始化
                nn.init.xavier_uniform_(module.weight)  # PyTorch的设计体系中，函数名末尾的下划线表示这是一个"就地操作"（in-place operation），意思是它会直接修改传入的张量，而不是返回一个新的张量。
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)

    def forward(self, x):
        return self.network(x)

mlp = MLP(32, [128, 64, 32], 1)
x = torch.randn(1, 2, 32)
print(x)
y = mlp(x)
print(y)


tensor([[[ 2.0681,  1.5619,  0.0909, -0.9272, -0.8474, -1.1844,  0.3646,
          -0.8629, -1.4969,  0.1424, -0.8461,  0.8752, -0.1210,  1.0228,
          -0.0918, -1.0681,  2.0352,  0.7658,  0.7031,  0.6318,  1.5409,
          -0.9169,  0.3927, -1.3012,  0.5303, -1.2943, -1.3301, -0.8466,
           2.1570,  0.6033,  0.2530,  1.4443],
         [ 0.4359,  0.5136,  1.7633, -1.3619,  0.7331,  0.5869, -0.7370,
          -0.1460,  0.8217,  0.5016, -0.6113,  1.1776,  0.4734,  1.2617,
          -0.3058,  0.2861, -1.4580, -0.0126, -0.3054,  1.1936,  1.0763,
           1.4527,  0.2317,  0.1980, -0.0380, -0.1888,  0.3183, -0.8052,
          -1.8534,  0.8622, -0.7915, -1.5688]]])
tensor([[[-0.8437],
         [-0.1871]]], grad_fn=<ViewBackward0>)


## 混合精度训练AMP

In [35]:
import torch 
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 1.基础配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16

# 2.数据准备
# transforms.ToTensor:()第一，数据类型转换(numpy或者image数据转为tensor)；第二，数值归一化。将0-255范围的像素值缩放到0-1范围；第三，维度重排[通道数，高度，宽度]
# Compose将多个transform串联成一个处理链，数据依次通过每个环节。
transform = transforms.Compose([transforms.ToTensor()]) 
train_dataset = datasets.MNIST(root = r'D:\科研\搜广推\04_手撕算法题\大模型_推荐算法_手撕题\data', train=True, download=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, pin_memory=True, num_workers=4)

# 3.模型
class simplemodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),  
            nn.Linear(512, 256),
            nn.ReLU(),  
            nn.Linear(256, 10)
        )
    
    def forward(self, x):
        return self.net(x)

# 损失函数、优化器等设置
"""
需要移动到 GPU 的是参与大规模计算的张量数据，包括模型参数和输入数据
损失函数和优化器是操作这些张量的“工具”，它们会自动在张量所在的设备上执行操作，所以不需要手动移动
"""
model = simplemodel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 4.训练设置
"""
训练模式
Dropout 层会按照设定的概率 p 随机地将一部分神经元的输出置为零。这是一种有效的正则化手段，可以防止模型过拟合
BN 层会计算当前批次数据的均值和方差，并用它们来归一化数据。同时，它还会维护一个全局的均值和方差，这个全局值是根据训练过程中所有批次的统计数据通过滑动平均更新的

推理模式
Dropout 层会“关闭”，不再随机丢弃任何神经元，而是让所有神经元的输出都通过。这样可以保证在预测时模型的输出是确定和稳定的
BN 层会“冻结”，不再计算当前批次的均值和方差，而是直接使用在整个训练集上学习到的全局均值和方差来进行归一化。这保证了在推理时，即使输入只有一个样本，也能得到一致和稳定的结果
"""

model.train()
def main():
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        optimizer.zero_grad()
        with torch.autocast("cuda", dtype=dtype):
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        
        loss.backward()     # PyTorch 会计算出模型中每个参数的梯度,默认情况下，这些新计算出的梯度会加到该参数已有的梯度上（如果已有梯度存在的话），而不是覆盖它们
        optimizer.step()    # 更新梯度
        if batch_idx % 10 == 0:
            print(f"epoch[{1}], step[{batch_idx}], loss:{loss.item()}")

if __name__ == '__main__':
    main()


epoch[1], step[0], loss:2.30328369140625
epoch[1], step[10], loss:2.25201416015625
epoch[1], step[20], loss:2.155517578125
epoch[1], step[30], loss:1.97998046875
epoch[1], step[40], loss:1.7359619140625
epoch[1], step[50], loss:1.3389091491699219
epoch[1], step[60], loss:0.8904438018798828
epoch[1], step[70], loss:0.7717809677124023
epoch[1], step[80], loss:0.7068207263946533
epoch[1], step[90], loss:0.5415992736816406
epoch[1], step[100], loss:0.4719715714454651
epoch[1], step[110], loss:0.47473612427711487
epoch[1], step[120], loss:0.37358441948890686
epoch[1], step[130], loss:0.4280766248703003
epoch[1], step[140], loss:0.42194801568984985
epoch[1], step[150], loss:0.3845781683921814
epoch[1], step[160], loss:0.29847246408462524
epoch[1], step[170], loss:0.41989171504974365
epoch[1], step[180], loss:0.36476024985313416
epoch[1], step[190], loss:0.32573196291923523
epoch[1], step[200], loss:0.3932267427444458
epoch[1], step[210], loss:0.3183577060699463
epoch[1], step[220], loss:0.29